# Import & examine data

In [1]:
import pandas as pd
import os
from pathlib import Path

cwd = Path(os.getcwd())
cmts_csv = cwd / 'Data' / 'Comments_20210607_234843.csv'
posts_csv = cwd / 'Data' / 'Posts_20210607_234843.csv'

cmts_df = pd.read_csv(cmts_csv,sep=',',index_col=0)
cmts_df.head(15)

,comment_id,post_id,body,sub,score,post_title,post_flair
0,h0zq22r,nuvc6e,Hi everyone! Here's a conpendium with seven po...,UnearthedArcana,1,The Seven Dragon Overlords - A Conpendium of g...,Compendium
1,h0xze5l,nukr9d,sentid0 has made the following comment(s) rega...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass
2,h0xzcqe,nukr9d,[https://homebrewery.naturalcrit.com/share/5pA...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass
3,h0xwncu,nukb9t,Hi everyone! Here’s a project that u/ShalkenTa...,UnearthedArcana,1,Empowered Sorcerer - Unleash the true potentia...,Class
4,h0xofx2,nuisto,I designed this weapon along with a subclass I...,UnearthedArcana,1,[Weapon] Great Shark Blade - Very Rare Macuahu...,Item
5,h0wd0mk,nuabf7,Hello all! I have been working on this homebre...,UnearthedArcana,1,War Domain Revised,Subclass
6,h0uca2o,ntwzm9,Homebrewery Link - https://homebrewery.natural...,UnearthedArcana,1,Sceptor of the Beyond - mess with your players...,Item
7,h0t7nll,ntpza5,The Apossi!\n\nThis a project I have been work...,UnearthedArcana,2,The Apossi - A race of spacefaring opossum peo...,Race
8,h0rq1lt,ntffqj,Homebrew full caster class created for my D&D ...,UnearthedArcana,1,Sonumancer - A supportive full caster class (a...,Class
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass


### Exploring comments...
Let's print the first 10 comments in full to see how it looks

In [2]:
for i in cmts_df['body'][0:9]:
    print(i+"\n")

   everyone! Here's a conpendium with seven powerful, fearsome dragons for your high-level campaign! This is an attempt at creating monsters powerful enough to serve as effective solo bosses, given how common it is for this type of encounters to fall under the power of action economy jaja
  
  ey're probably a bit overtuned, but they are high CR monsters intended to be thrown at players at a lever where game balance goes out of the window, so I had some fun imagining and creating this thematic dragons with fun abilities and give them interesting mechanics to interact with your players.
  
   a bonus, there's some lore and history about the campaign where they'll inaugurate, as well as their personality and interactions with other dragons.
  
   course, all art credit is at the last page, and my personal thanks to my girlfriend who helped me edit this.
  
  pe you like this conpendium! If you have any opinions , questions or feedback, I'll be eagerly reading them in the comments below!


# Extracting the GMBinder and/or Homebrewery links

We can see we have some different varieties of potentially useful links, including /share/ and /edit/ links. There are also some from unrelated domains (Google Docs, Wikipedia articles, etc.) and some direct links to PDF files that lack the HTML source we want to collect.

We also have three varieties of link text:

## Raw links
URLs with no special formating around them.
> <pre>Gmbinder Link: https://www.gmbinder.com/share/-MW5xKc1QPj4TXEgBzQm</pre>

## Markdown links
These take the form of a display text portion in brackets immediately followed by a URL in parenthesis.
> <pre>[Display text](url)</pre>

Moreover, in some cases, the display text is also a url and in at least one case early on the url is incorrect in the display text portion, but correct in the actual url segment of the link code.

> <pre>[https://homebrewery.naturalcrit.com/share/MhTK9\_7zHuB6](https://homebrewery.naturalcrit.com/share/MhTK9_7zHuB6)</pre>

Since, the url portion is more likely to be correct in most cases, we'll go exclusively with raw links and the (url) portion of markdown code.

To do this we can use a set of regular expressions. After a few trial and errors, I arrived at the following:

### Homebrewery
> <pre>?:https?:\/\/)?(?:www\.)?homebrewery\.naturalcrit\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*</pre>

### GMBinder
> <pre>(?:https?:\/\/)?(?:www\.)?gmbinder\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\)]</pre>

In [3]:
cmts_df['url'] = cmts_df['body'].str.findall(r'[\s\(](?:https?:\/\/)?(?:www\.)?homebrewery\.naturalcrit\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\\)]|[\s\(](?:https?:\/\/)?(?:www\.)?gmbinder\.com\/[\w\-][\w\-]*\/?[\w\-]*\/?[\w\-]*[\s\)]')
for i in cmts_df['url'][0:14]:
    print(i)

['(https://www.gmbinder.com/share/-Ma-uM6qJVQrOQCqAW_h)']
[]
['(https://homebrewery.naturalcrit.com/share/5pAZ7yalcycb)']
['(https://www.gmbinder.com/share/-MbXDsntDZfWFZTSvI1u)']
['(https://www.gmbinder.com/share/-MY93ONkI97gQojUCamq)']
['(https://homebrewery.naturalcrit.com/share/MuXv0wp7T3VP)']
[' https://homebrewery.naturalcrit.com/share/bwlZ\\']
['(https://homebrewery.naturalcrit.com/share/1akc6sy-yLCYkmdigekOAI7MeXX2vUvmxL2M60vRkIjRE)']
['(https://www.gmbinder.com/share/-Mb0TEktVmDXVBJp1o75)']
['(https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs10J8B)', '(https://www.gmbinder.com/share/-MaFl4cOGqS5DpWWlGK_)']
[]
['(https://www.gmbinder.com/share/-MaGpUOUtN5708lGugMr)']
['(https://www.gmbinder.com/share/-MaiiTks5XL8QJDoreAT)']
['(https://homebrewery.naturalcrit.com/share/1PKVJd1CVvEvSPUJtF3Y0f2lr_G60Sw96EbCjzmP6VhS0)']


# Final cleaning steps...

As expected, some entries have multiple urls, while others now lack any entries.

Many of our strings also having some hanging portions at the ends from our Regex expression:
* '('
* ')'
* '\n'
* ' '

For our final data, we'd like to get everything in a format where each row represets one URL rather than one post and each URL is unique and has been cleaned.

Lastly, we'll remove anything with 'profile' or 'pdf' in the url, as these URLs require significantly more processing to handle and would not be worth the effort (especially the PDFs)

Luckily, we have a few convient tools at our disposal, so we will takle this in the following manner:

1) Expand each entry into multiple rows for each URL in the dataset. To do this we will use Pandas' .explode() function designed for this very purpose.

2) Remove rows that are now empty (i.e. posts that only contained direct links to the homebrewery/gmbinder homepage) by dropping NA values

3) Each each individual string to remove the extraneous portions left over using the map function.

4) Remove duplicates using the drop_duplicates() function in Pandas.

5) Lastly, we'll remove any rows that contain the string 'pdf' or 'profile'

In [4]:
cmts_df = cmts_df.explode('url')
cmts_df = cmts_df.dropna()
cmts_df['url'] = cmts_df['url'].map(lambda x : x.strip('(').strip(')').strip('\n').strip(' '))
cmts_df.drop_duplicates(subset=['url'],keep='first',inplace=True)
cmts_df = cmts_df[cmts_df['url'].str.contains('pdf') == False]
cmts_df = cmts_df[cmts_df['url'].str.contains('profile') == False]
cmts_df = cmts_df[cmts_df['url'].str.contains('edit') == False]
cmts_df.reindex()
cmts_df.head(25)

,comment_id,post_id,body,sub,score,post_title,post_flair,url
0,h0zq22r,nuvc6e,Hi everyone! Here's a conpendium with seven po...,UnearthedArcana,1,The Seven Dragon Overlords - A Conpendium of g...,Compendium,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...
2,h0xzcqe,nukr9d,[https://homebrewery.naturalcrit.com/share/5pA...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass,https://homebrewery.naturalcrit.com/share/5pAZ...
3,h0xwncu,nukb9t,Hi everyone! Here’s a project that u/ShalkenTa...,UnearthedArcana,1,Empowered Sorcerer - Unleash the true potentia...,Class,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...
4,h0xofx2,nuisto,I designed this weapon along with a subclass I...,UnearthedArcana,1,[Weapon] Great Shark Blade - Very Rare Macuahu...,Item,https://www.gmbinder.com/share/-MY93ONkI97gQoj...
5,h0wd0mk,nuabf7,Hello all! I have been working on this homebre...,UnearthedArcana,1,War Domain Revised,Subclass,https://homebrewery.naturalcrit.com/share/MuXv...
6,h0uca2o,ntwzm9,Homebrewery Link - https://homebrewery.natural...,UnearthedArcana,1,Sceptor of the Beyond - mess with your players...,Item,https://homebrewery.naturalcrit.com/share/bwlZ\
7,h0t7nll,ntpza5,The Apossi!\n\nThis a project I have been work...,UnearthedArcana,2,The Apossi - A race of spacefaring opossum peo...,Race,https://homebrewery.naturalcrit.com/share/1akc...
8,h0rq1lt,ntffqj,Homebrew full caster class created for my D&D ...,UnearthedArcana,1,Sonumancer - A supportive full caster class (a...,Class,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...


# Collect Markdown/Source Code

Next, we'll import the post data and collect the mixed html/css/markdown-style source code used by Homebrewery/GMBinder.

For this, I've defined two functions in dnd_scraper_tools that I'll be importing.

The first function simply checks for a source code button. This seems to be standard on Homebrewery, but can be optionally disabled on GMBinder.

In that event, we'll try to get the cleanest text we can from the HTML available on the main display page, so the function will return the original URL if it can't find a button. This is done only for the GMBinder links, as Homebrewery's mcontained directain display page is rendered by React JS and collecting the text via BeautifulSoup is not possible. My initial exploration of the links suggests that this does not occur. If it does, the script will return an value of None that we can later filter out.

##### Note, I performed some manual curation of the post data to add UnearthedArcana style descriptive flair tags to the DnDHomebrew posts, that way this datset is initially complete with respect to that column.

In [5]:
from dnd_scraper_tools import grab_src_url, collect_text, remove_html

posts_df = pd.read_csv(posts_csv,sep=',',index_col=0)

posts_df = posts_df[posts_df['url'].str.contains('pdf') == False]
posts_df = posts_df[posts_df['url'].str.contains('profile') == False]
posts_df = posts_df[posts_df['url'].str.contains('edit') == False]

In [6]:
posts_df.head(10)

,post_id,title,url,sub,score,flair,upvote_ratio
0,nun32b,"The Charge Blade, v1.26: A powerful switch-wea...",https://homebrewery.naturalcrit.com/share/8wjc...,UnearthedArcana,4,Item,0.84
1,nu968t,Way of the Five Animals [1.1],https://homebrewery.naturalcrit.com/share/GcYx...,UnearthedArcana,2,Subclass,1.00
2,ntpgqq,Weapon Master Martial Archetype: A walking ars...,https://homebrewery.naturalcrit.com/share/14kh...,UnearthedArcana,7,Subclass,1.00
3,ntn4qt,Heroes of the Old West: Some new subclasses fo...,https://homebrewery.naturalcrit.com/share/77Z7...,UnearthedArcana,3,Subclass,0.80
4,nt9d4y,"(2/13) The Barbarian, Unchained - A revised ve...",https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,UnearthedArcana,11,Class,0.87
5,nt4dck,"Way of the Nine Gates, Version 4 - Unleash you...",https://homebrewery.naturalcrit.com/share/vvoj...,UnearthedArcana,5,Subclass,0.79
6,nseeyi,Jester 0.7 | Versatile class with 5 subclasses...,https://homebrewery.naturalcrit.com/share/9Kyr...,UnearthedArcana,5,Class,0.86
7,nrregt,In my homebrew Setting Oozes are considered a ...,https://homebrewery.naturalcrit.com/share/tCSX...,UnearthedArcana,11,Subclass,1.00
8,nqpz6b,Archon Soul - A Sorcerer subclass designed aro...,https://homebrewery.naturalcrit.com/share/HCoa...,UnearthedArcana,8,Subclass,0.83
9,nqgykq,"Made my own Vestige of Divergence. ""The Redemp...",https://homebrewery.naturalcrit.com/share/1t67...,UnearthedArcana,12,Item,0.93


In [7]:
posts_df['src_url'] = posts_df['url'].map(grab_src_url)
posts_df.head(10)

,post_id,title,url,sub,score,flair,upvote_ratio,src_url
0,nun32b,"The Charge Blade, v1.26: A powerful switch-wea...",https://homebrewery.naturalcrit.com/share/8wjc...,UnearthedArcana,4,Item,0.84,https://homebrewery.naturalcrit.com/source/8wj...
1,nu968t,Way of the Five Animals [1.1],https://homebrewery.naturalcrit.com/share/GcYx...,UnearthedArcana,2,Subclass,1.00,https://homebrewery.naturalcrit.com/source/GcY...
2,ntpgqq,Weapon Master Martial Archetype: A walking ars...,https://homebrewery.naturalcrit.com/share/14kh...,UnearthedArcana,7,Subclass,1.00,https://homebrewery.naturalcrit.com/source/14k...
3,ntn4qt,Heroes of the Old West: Some new subclasses fo...,https://homebrewery.naturalcrit.com/share/77Z7...,UnearthedArcana,3,Subclass,0.80,https://homebrewery.naturalcrit.com/source/77Z...
4,nt9d4y,"(2/13) The Barbarian, Unchained - A revised ve...",https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,UnearthedArcana,11,Class,0.87,https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...
5,nt4dck,"Way of the Nine Gates, Version 4 - Unleash you...",https://homebrewery.naturalcrit.com/share/vvoj...,UnearthedArcana,5,Subclass,0.79,https://homebrewery.naturalcrit.com/source/vvo...
6,nseeyi,Jester 0.7 | Versatile class with 5 subclasses...,https://homebrewery.naturalcrit.com/share/9Kyr...,UnearthedArcana,5,Class,0.86,https://homebrewery.naturalcrit.com/source/9Ky...
7,nrregt,In my homebrew Setting Oozes are considered a ...,https://homebrewery.naturalcrit.com/share/tCSX...,UnearthedArcana,11,Subclass,1.00,https://homebrewery.naturalcrit.com/source/tCS...
8,nqpz6b,Archon Soul - A Sorcerer subclass designed aro...,https://homebrewery.naturalcrit.com/share/HCoa...,UnearthedArcana,8,Subclass,0.83,https://homebrewery.naturalcrit.com/source/HCo...
9,nqgykq,"Made my own Vestige of Divergence. ""The Redemp...",https://homebrewery.naturalcrit.com/share/1t67...,UnearthedArcana,12,Item,0.93,https://homebrewery.naturalcrit.com/source/1t6...


In [8]:
cmts_df['src_url'] = cmts_df['url'].map(grab_src_url)
cmts_df.head(10)

,comment_id,post_id,body,sub,score,post_title,post_flair,url,src_url
0,h0zq22r,nuvc6e,Hi everyone! Here's a conpendium with seven po...,UnearthedArcana,1,The Seven Dragon Overlords - A Conpendium of g...,Compendium,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...
2,h0xzcqe,nukr9d,[https://homebrewery.naturalcrit.com/share/5pA...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass,https://homebrewery.naturalcrit.com/share/5pAZ...,https://homebrewery.naturalcrit.com/source/5pA...
3,h0xwncu,nukb9t,Hi everyone! Here’s a project that u/ShalkenTa...,UnearthedArcana,1,Empowered Sorcerer - Unleash the true potentia...,Class,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...
4,h0xofx2,nuisto,I designed this weapon along with a subclass I...,UnearthedArcana,1,[Weapon] Great Shark Blade - Very Rare Macuahu...,Item,https://www.gmbinder.com/share/-MY93ONkI97gQoj...,https://www.gmbinder.com/share/-MY93ONkI97gQoj...
5,h0wd0mk,nuabf7,Hello all! I have been working on this homebre...,UnearthedArcana,1,War Domain Revised,Subclass,https://homebrewery.naturalcrit.com/share/MuXv...,https://homebrewery.naturalcrit.com/source/MuX...
6,h0uca2o,ntwzm9,Homebrewery Link - https://homebrewery.natural...,UnearthedArcana,1,Sceptor of the Beyond - mess with your players...,Item,https://homebrewery.naturalcrit.com/share/bwlZ\,https://homebrewery.naturalcrit.com/share/bwlZ\
7,h0t7nll,ntpza5,The Apossi!\n\nThis a project I have been work...,UnearthedArcana,2,The Apossi - A race of spacefaring opossum peo...,Race,https://homebrewery.naturalcrit.com/share/1akc...,https://homebrewery.naturalcrit.com/source/1ak...
8,h0rq1lt,ntffqj,Homebrew full caster class created for my D&D ...,UnearthedArcana,1,Sonumancer - A supportive full caster class (a...,Class,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...


In [9]:
cmts_df['Text'] = cmts_df['src_url'].map(collect_text)
posts_df['Text'] = posts_df['src_url'].map(collect_text)

In [10]:
cmts_df.head(10)

,comment_id,post_id,body,sub,score,post_title,post_flair,url,src_url,Text
0,h0zq22r,nuvc6e,Hi everyone! Here's a conpendium with seven po...,UnearthedArcana,1,The Seven Dragon Overlords - A Conpendium of g...,Compendium,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...,<div class='cover-diamond'></div>\n\n<div clas...
2,h0xzcqe,nukr9d,[https://homebrewery.naturalcrit.com/share/5pA...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass,https://homebrewery.naturalcrit.com/share/5pAZ...,https://homebrewery.naturalcrit.com/source/5pA...,<img \n src='https://i.imgur.com/TeZRWAP.png'...
3,h0xwncu,nukb9t,Hi everyone! Here’s a project that u/ShalkenTa...,UnearthedArcana,1,Empowered Sorcerer - Unleash the true potentia...,Class,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...,\n \n \n \n \n \n \n \n ...
4,h0xofx2,nuisto,I designed this weapon along with a subclass I...,UnearthedArcana,1,[Weapon] Great Shark Blade - Very Rare Macuahu...,Item,https://www.gmbinder.com/share/-MY93ONkI97gQoj...,https://www.gmbinder.com/share/-MY93ONkI97gQoj...,<style>\n .phb {width: 500px;height: auto;}\n...
5,h0wd0mk,nuabf7,Hello all! I have been working on this homebre...,UnearthedArcana,1,War Domain Revised,Subclass,https://homebrewery.naturalcrit.com/share/MuXv...,https://homebrewery.naturalcrit.com/source/MuX...,## Cleric: War Domain Revised\nWar has many ma...
6,h0uca2o,ntwzm9,Homebrewery Link - https://homebrewery.natural...,UnearthedArcana,1,Sceptor of the Beyond - mess with your players...,Item,https://homebrewery.naturalcrit.com/share/bwlZ\,https://homebrewery.naturalcrit.com/share/bwlZ\,Can not find brew
7,h0t7nll,ntpza5,The Apossi!\n\nThis a project I have been work...,UnearthedArcana,2,The Apossi - A race of spacefaring opossum peo...,Race,https://homebrewery.naturalcrit.com/share/1akc...,https://homebrewery.naturalcrit.com/source/1ak...,"## Apossi\nIT WAS LARGE, LARGER THAN ANY FLEET..."
8,h0rq1lt,ntffqj,Homebrew full caster class created for my D&D ...,UnearthedArcana,1,Sonumancer - A supportive full caster class (a...,Class,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...,<style>\n <style>\n/* Background */\n .phb{ ...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...,\n \n \n \n \n \n \n \n ...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...,\n \n \n \n \n \n \n \n ...


In [11]:
posts_df.head(10)

,post_id,title,url,sub,score,flair,upvote_ratio,src_url,Text
0,nun32b,"The Charge Blade, v1.26: A powerful switch-wea...",https://homebrewery.naturalcrit.com/share/8wjc...,UnearthedArcana,4,Item,0.84,https://homebrewery.naturalcrit.com/source/8wj...,# The Charge Blade <sup>v1.26</sup>\nWhen faci...
1,nu968t,Way of the Five Animals [1.1],https://homebrewery.naturalcrit.com/share/GcYx...,UnearthedArcana,2,Subclass,1.00,https://homebrewery.naturalcrit.com/source/GcY...,# Monastic Order v1.1\n\n## Way of the Five An...
2,ntpgqq,Weapon Master Martial Archetype: A walking ars...,https://homebrewery.naturalcrit.com/share/14kh...,UnearthedArcana,7,Subclass,1.00,https://homebrewery.naturalcrit.com/source/14k...,### Weapon Master Martial Archetype\n\nAll fig...
3,ntn4qt,Heroes of the Old West: Some new subclasses fo...,https://homebrewery.naturalcrit.com/share/77Z7...,UnearthedArcana,3,Subclass,0.80,https://homebrewery.naturalcrit.com/source/77Z...,<style>\n .phb#p1{ text-align:center; }\n .p...
4,nt9d4y,"(2/13) The Barbarian, Unchained - A revised ve...",https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,UnearthedArcana,11,Class,0.87,https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,\n \n \n \n \n \n \n \n ...
5,nt4dck,"Way of the Nine Gates, Version 4 - Unleash you...",https://homebrewery.naturalcrit.com/share/vvoj...,UnearthedArcana,5,Subclass,0.79,https://homebrewery.naturalcrit.com/source/vvo...,"# Way of the Nine Gates\n\n*“ Once, I traveled..."
6,nseeyi,Jester 0.7 | Versatile class with 5 subclasses...,https://homebrewery.naturalcrit.com/share/9Kyr...,UnearthedArcana,5,Class,0.86,https://homebrewery.naturalcrit.com/source/9Ky...,"# The Jester, 5e\n\n<div class='wide'>\n\n####..."
7,nrregt,In my homebrew Setting Oozes are considered a ...,https://homebrewery.naturalcrit.com/share/tCSX...,UnearthedArcana,11,Subclass,1.00,https://homebrewery.naturalcrit.com/source/tCS...,## Pact of the Dire Ooze\nVery few Oozes can g...
8,nqpz6b,Archon Soul - A Sorcerer subclass designed aro...,https://homebrewery.naturalcrit.com/share/HCoa...,UnearthedArcana,8,Subclass,0.83,https://homebrewery.naturalcrit.com/source/HCo...,### Archon Soul\nYour innate magic is derived ...
9,nqgykq,"Made my own Vestige of Divergence. ""The Redemp...",https://homebrewery.naturalcrit.com/share/1t67...,UnearthedArcana,12,Item,0.93,https://homebrewery.naturalcrit.com/source/1t6...,# The Redemptor V1.1\n\nWeapon (hand crossbow)...


# Removing residual HTML
A quick scan reveals that some HTML tags in the source text (extra styling elements, etc) did not get removed. Let's strip all of the HTML one more time.

In [12]:
cmts_df['Text'] = cmts_df['Text'].map(remove_html)
posts_df['Text'] = posts_df['Text'].map(remove_html)

In [13]:
posts_df.head(10)

,post_id,title,url,sub,score,flair,upvote_ratio,src_url,Text
0,nun32b,"The Charge Blade, v1.26: A powerful switch-wea...",https://homebrewery.naturalcrit.com/share/8wjc...,UnearthedArcana,4,Item,0.84,https://homebrewery.naturalcrit.com/source/8wj...,# The Charge Blade v1.26\nWhen facing down the...
1,nu968t,Way of the Five Animals [1.1],https://homebrewery.naturalcrit.com/share/GcYx...,UnearthedArcana,2,Subclass,1.00,https://homebrewery.naturalcrit.com/source/GcY...,# Monastic Order v1.1\n\n## Way of the Five An...
2,ntpgqq,Weapon Master Martial Archetype: A walking ars...,https://homebrewery.naturalcrit.com/share/14kh...,UnearthedArcana,7,Subclass,1.00,https://homebrewery.naturalcrit.com/source/14k...,### Weapon Master Martial Archetype\n\nAll fig...
3,ntn4qt,Heroes of the Old West: Some new subclasses fo...,https://homebrewery.naturalcrit.com/share/77Z7...,UnearthedArcana,3,Subclass,0.80,https://homebrewery.naturalcrit.com/source/77Z...,\n .phb#p1{ text-align:center; }\n .phb#p1:a...
4,nt9d4y,"(2/13) The Barbarian, Unchained - A revised ve...",https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,UnearthedArcana,11,Class,0.87,https://www.gmbinder.com/share/-MaJ6FkbPNXZwc3...,Unchained Barbarian\n by Au...
5,nt4dck,"Way of the Nine Gates, Version 4 - Unleash you...",https://homebrewery.naturalcrit.com/share/vvoj...,UnearthedArcana,5,Subclass,0.79,https://homebrewery.naturalcrit.com/source/vvo...,"# Way of the Nine Gates\n\n*“ Once, I traveled..."
6,nseeyi,Jester 0.7 | Versatile class with 5 subclasses...,https://homebrewery.naturalcrit.com/share/9Kyr...,UnearthedArcana,5,Class,0.86,https://homebrewery.naturalcrit.com/source/9Ky...,"# The Jester, 5e\n\n\n\n###### *""A fugitive ta..."
7,nrregt,In my homebrew Setting Oozes are considered a ...,https://homebrewery.naturalcrit.com/share/tCSX...,UnearthedArcana,11,Subclass,1.00,https://homebrewery.naturalcrit.com/source/tCS...,## Pact of the Dire Ooze\nVery few Oozes can g...
8,nqpz6b,Archon Soul - A Sorcerer subclass designed aro...,https://homebrewery.naturalcrit.com/share/HCoa...,UnearthedArcana,8,Subclass,0.83,https://homebrewery.naturalcrit.com/source/HCo...,### Archon Soul\nYour innate magic is derived ...
9,nqgykq,"Made my own Vestige of Divergence. ""The Redemp...",https://homebrewery.naturalcrit.com/share/1t67...,UnearthedArcana,12,Item,0.93,https://homebrewery.naturalcrit.com/source/1t6...,# The Redemptor V1.1\n\nWeapon (hand crossbow)...


In [14]:
cmts_df.head(10)

,comment_id,post_id,body,sub,score,post_title,post_flair,url,src_url,Text
0,h0zq22r,nuvc6e,Hi everyone! Here's a conpendium with seven po...,UnearthedArcana,1,The Seven Dragon Overlords - A Conpendium of g...,Compendium,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...,https://www.gmbinder.com/share/-Ma-uM6qJVQrOQC...,\n\nThe Seven Dragon Overlords\n\n\n\n A Draco...
2,h0xzcqe,nukr9d,[https://homebrewery.naturalcrit.com/share/5pA...,UnearthedArcana,1,Ranger: Mage Slayer v2 (Art by Johnathan),Subclass,https://homebrewery.naturalcrit.com/share/5pAZ...,https://homebrewery.naturalcrit.com/source/5pA...,\n\n\n\n## Ranger: Mage Slayer\nYou are the su...
3,h0xwncu,nukb9t,Hi everyone! Here’s a project that u/ShalkenTa...,UnearthedArcana,1,Empowered Sorcerer - Unleash the true potentia...,Class,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...,https://www.gmbinder.com/share/-MbXDsntDZfWFZT...,Empowered Sorcerer\n by Vos...
4,h0xofx2,nuisto,I designed this weapon along with a subclass I...,UnearthedArcana,1,[Weapon] Great Shark Blade - Very Rare Macuahu...,Item,https://www.gmbinder.com/share/-MY93ONkI97gQoj...,https://www.gmbinder.com/share/-MY93ONkI97gQoj...,\n .phb {width: 500px;height: auto;}\n .phb:...
5,h0wd0mk,nuabf7,Hello all! I have been working on this homebre...,UnearthedArcana,1,War Domain Revised,Subclass,https://homebrewery.naturalcrit.com/share/MuXv...,https://homebrewery.naturalcrit.com/source/MuX...,## Cleric: War Domain Revised\nWar has many ma...
6,h0uca2o,ntwzm9,Homebrewery Link - https://homebrewery.natural...,UnearthedArcana,1,Sceptor of the Beyond - mess with your players...,Item,https://homebrewery.naturalcrit.com/share/bwlZ\,https://homebrewery.naturalcrit.com/share/bwlZ\,Can not find brew
7,h0t7nll,ntpza5,The Apossi!\n\nThis a project I have been work...,UnearthedArcana,2,The Apossi - A race of spacefaring opossum peo...,Race,https://homebrewery.naturalcrit.com/share/1akc...,https://homebrewery.naturalcrit.com/source/1ak...,"## Apossi\nIT WAS LARGE, LARGER THAN ANY FLEET..."
8,h0rq1lt,ntffqj,Homebrew full caster class created for my D&D ...,UnearthedArcana,1,Sonumancer - A supportive full caster class (a...,Class,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...,https://www.gmbinder.com/share/-Mb0TEktVmDXVBJ...,\n <style>\n/* Background */\n .phb{ backgro...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...,https://www.gmbinder.com/share/-MaUdYa0vZ3L5rs...,Roguish Archetype: The Fearful\n ...
9,h0rbo6s,ntccfa,Hello it's me again!\n\nI'm back with the upda...,UnearthedArcana,1,Subclasses | Fighter: Arcane Godfather (v1.1) ...,Subclass,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...,https://www.gmbinder.com/share/-MaFl4cOGqS5DpW...,Martial Archetype: Arcane Godfather\n ...


In [15]:
processed_cmts = cwd / 'Data' / 'CommentsProcessed.csv'
processed_posts = cwd / 'Data' / 'PostsProcessed.csv'

posts_df.to_csv(processed_posts,sep=',',encoding='utf-8')
cmts_df.to_csv(processed_cmts,sep=',',encoding='utf-8')

In [16]:
cmts_df.rename(columns={"post_flair":"flair"},inplace=True)
cmts_df.rename(columns={"post_title":"title"},inplace=True)

In [17]:
all_data = pd.concat([posts_df[["src_url","sub","flair","title","Text"]], cmts_df[["src_url","sub","flair","title","Text"]]])
all_data.drop_duplicates(subset=["src_url"], inplace=True)

In [18]:
processed = cwd / 'Data' / 'TestProcessed.csv'

all_data = all_data[["sub","flair","title","Text","src_url"]]

all_data.to_csv(processed, sep=',', index=False,encoding='utf-8')